In [21]:
!uv --version

uv 0.9.5 (d5f39331a 2025-10-21)


In [20]:
import tomllib  # built into Python 3.11+

with open('./uv.lock', 'rb') as lockfile:  # must open in binary mode for tomllib
    uv_data = tomllib.load(lockfile)

# Let's inspect the keys to understand the structure
for key in uv_data.keys():
    print(key)

# To find scikit-learn, loop through the 'package' list
for pkg in uv_data.get('package', []):
    if pkg.get('name') == 'scikit-learn':
        print("Found scikit-learn:")
        print(pkg)
        break
else:
    print("scikit-learn not found in uv.lock")

version
revision
requires-python
package
Found scikit-learn:
{'name': 'scikit-learn', 'version': '1.7.2', 'source': {'registry': 'https://pypi.org/simple'}, 'dependencies': [{'name': 'joblib'}, {'name': 'numpy'}, {'name': 'scipy'}, {'name': 'threadpoolctl'}], 'sdist': {'url': 'https://files.pythonhosted.org/packages/98/c2/a7855e41c9d285dfe86dc50b250978105dce513d6e459ea66a6aeb0e1e0c/scikit_learn-1.7.2.tar.gz', 'hash': 'sha256:20e9e49ecd130598f1ca38a1d85090e1a600147b9c02fa6f15d69cb53d968fda', 'size': 7193136, 'upload-time': '2025-09-09T08:21:29.075Z'}, 'wheels': [{'url': 'https://files.pythonhosted.org/packages/d9/82/dee5acf66837852e8e68df6d8d3a6cb22d3df997b733b032f513d95205b7/scikit_learn-1.7.2-cp314-cp314-macosx_10_13_x86_64.whl', 'hash': 'sha256:fa8f63940e29c82d1e67a45d5297bdebbcb585f5a5a50c4914cc2e852ab77f33', 'size': 9208906, 'upload-time': '2025-09-09T08:21:18.557Z'}, {'url': 'https://files.pythonhosted.org/packages/3c/30/9029e54e17b87cb7d50d51a5926429c683d5b4c1732f0507a6c3bed9bf65

In [16]:
import pickle
import warnings

warnings.filterwarnings("ignore")

# Load the pipeline model
with open('pipeline_v1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

# Example record
lead = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

# Get the probability of conversion
pred_proba = model.predict_proba([lead])[0, 1]
print(f"Lead conversion probability: {pred_proba:.3f}")

Lead conversion probability: 0.534


In [17]:
import pickle
from fastapi import FastAPI
from pydantic import BaseModel

# Load the pipeline model
with open('pipeline_v1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

# Define input schema
class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

# Create FastAPI app
app = FastAPI()

# Prediction endpoint
@app.post("/predict")
def predict(client: Client):
    data = client.dict()
    proba = model.predict_proba([data])[0, 1]
    return {"subscription_probability": round(float(proba), 3)}

In [19]:
import requests

url = "http://127.0.0.1:8000/predict"

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())

{'subscription_probability': 0.534}
